In [1]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta

import requests

In [22]:
def special_characters(city):
    if city == 'Munchen':
        return 'München'
    elif city == 'Koln':
        return 'Köln'
    elif city == 'Wurzburg':
        return 'Würzburg'
    elif city == 'Nurnberg':
        return 'Nürnberg'
    else:
        return city

def day_categories(day):
    time_of_day_cat = ['night', 'morning', 'afternoon', 'evening']
    if 0 < day.hour < 6:
        return time_of_day_cat[0]
    elif 6 < day.hour < 12:
        return time_of_day_cat[1]
    elif 12 < day.hour < 18:
        return time_of_day_cat[2]
    elif 18 < day.hour < 24:
        return time_of_day_cat[3]
    
def coco_func(x, coco_forecast, new_classes_forecast):
    x_list = x.split(', ')
    for k, v in coco_forecast.items():
        if v in x_list:
            for c, i in new_classes_forecast.items():
                if k in i:
                    return c

In [3]:
start_city = 'München'
end_city = 'Köln'
user_date = '2022-06-06 17:27'

In [33]:
start_city =  special_characters(start_city)
end_city = special_characters(end_city)

assert isinstance(user_date, str)

# get weather forecast
key = '7DYDYYY5GVYHQA52HXFQV5A5Y'

start_date = pd.Timestamp(user_date).round('H')
user_date_round_str = datetime.strftime(start_date, '%Y-%m-%d %H:%M')
end_date = start_date + timedelta(1)

start_date_str = datetime.strftime(start_date, '%Y-%m-%d')
end_date_str = datetime.strftime(end_date, '%Y-%m-%d')

weekday = start_date.weekday()
month = start_date.month

time_of_day = day_categories(start_date)

stations_lat_lon = pd.read_csv('../goodtrainbadtrain/data/Deutsche_Bahn_Haltestellen.csv', usecols=['X', 'Y', 'NAME'])
stations = ['Köln Hbf',
            'München Hbf',
            'Mannheim Hbf',
            'Stuttgart Hbf',
            'Würzburg Hbf',
            'Frankfurt(Main)Hbf',
            'Nürnberg Hbf'
            ]

coco_forecast = pd.read_csv('../goodtrainbadtrain/data/weather_coco_forecast.csv', sep=';')
coco_forecast.set_index('Code', inplace=True)
coco_forecast = coco_forecast.to_dict()['Weather Condition']
coco_forecast

new_classes_forecast = {
    'good': [29, 42, 43],
    'medium': [2, 8, 9, 19, 20, 21, 24, 27, 28, 30, 31, 32, 33, 36, 38, 39, 40, 41],
    'bad': [1, 4, 6, 11, 12, 14, 18, 23, 26, 35, 37],
    'extreme': [3, 5, 7, 10, 13, 15, 16, 17, 22, 25, 34]
}

weather_response = {}
weather = {}

for station in stations:
    lat = stations_lat_lon[stations_lat_lon['NAME'] == station]['Y'].mean()
    lon = stations_lat_lon[stations_lat_lon['NAME'] == station]['X'].mean()

    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{str(lat)},{str(lon)}/{start_date_str}/{end_date_str}"
    params = {'key': key,
                'unitGroup': 'metric'}
    response = requests.get(url, params=params).json()

    weather_response[station] = response

    days_list = pd.date_range(start_date, end_date)
    variables = ['datetime', 'temp', 'precip', 'snow', 'windspeed', 'windgust', 'conditions']
    sw_f = pd.DataFrame(columns=variables)
    for i, _ in enumerate(days_list):
        for h in range(24):
            date = response['days'][i]['datetime']
            hour = datetime.strftime(pd.to_datetime(h, format='%H'), '%H:%M:%S')
            date = date + ' ' + hour
            sw_f.loc[h + h*i + 24*i, 'datetime'] = date
            for v in variables[1:]:
                sw_f.loc[h + h*i + 24*i, v] = response['days'][i]['hours'][h][v]

    sw_f.rename(columns={'datetime': 'time',
                        'precip': 'prcp',
                        'windspeed': 'wspd',
                        'windgust': 'wpgt',
                        'conditions': 'coco'}, inplace=True)

    sw_f.set_index('time', inplace=True)

    weather[station] = sw_f.copy()

    weather[station]['coco'] = weather[station]['coco'].apply(lambda x: coco_func(x, coco_forecast, new_classes_forecast))
    weather[station].reset_index(inplace=True)
    weather[station]['time']  = pd.to_datetime(weather[station]['time'])

# Build prediction dataframe
X = pd.DataFrame({
    'trip': '-'.join([start_city, end_city]),
    'temp': weather[end_city + ' Hbf'][weather[end_city + ' Hbf']['time'] == user_date_round_str]['temp'],
    'prcp': weather[end_city + ' Hbf'][weather[end_city + ' Hbf']['time'] == user_date_round_str]['prcp'],
    'wspd': weather[end_city + ' Hbf'][weather[end_city + ' Hbf']['time'] == user_date_round_str]['wspd'],
    'wpgt': weather[end_city + ' Hbf'][weather[end_city + ' Hbf']['time'] == user_date_round_str]['wpgt'],
    'snow': weather[end_city + ' Hbf'][weather[end_city + ' Hbf']['time'] == user_date_round_str]['snow'],
    'coco': weather[end_city + ' Hbf'][weather[end_city + ' Hbf']['time'] == user_date_round_str]['coco'],
    'weekday': weekday,
    'month': month,
    'time_of_day':time_of_day
})

In [9]:
X

,trip,temp,prcp,wspd,wpgt,snow,coco,weekday,month,time_of_day
17,München-Köln,21.4,0.0,18.0,43.6,0.0,good,0,6,afternoon


In [ ]:
X = data[['trip', 'temp', 'prcp', 'wspd', 'wpgt', 'snow', 'coco', 'weekday', 'month', 'time_of_day']]

In [58]:
stations_test = ['München', 'Köln', 'Stuttgart', 'Frankfurt']

In [53]:
test = pd.DataFrame({
    'A': ['München', 'Köln', 'Stuttgart', 'Frankfurt'],
    'B': ['Köln', 'Frankfurt', 'München', 'Stuttgart'],
    'sh': ['a', 'b', 'c', 'd']
})

In [54]:
test_b = pd.DataFrame({
    'v1': [9, 10, 1, 2],
    'v2': [4, 6, -1, -2],
    'sh': ['a', 'b', 'c', 'd']
})

In [90]:
test

,A,B,sh
0,München,Köln,a
1,Köln,Frankfurt,b
2,Stuttgart,München,c
3,Frankfurt,Stuttgart,d


In [91]:
test_b

,v1,v2,sh
0,9,4,a
1,10,6,b
2,1,-1,c
3,2,-2,d


In [98]:
total_df =  pd.DataFrame()
st = 'Köln'
weather_df = pd.DataFrame(columns=['v1_oc, v2_oc, v1_dc, v2_dc'])
#weather_df = weather_df.append(test_b[test_b['sh'] == test['sh']])
oc_station_df  = pd.merge(test[test['A'] == st], test_b.add_suffix('_oc'), how='left', 
                            left_on='sh', right_on='sh_oc')
dc_station_df = pd.merge(test[test['B'] == st], test_b.add_suffix('_dc'), how='left', 
                            left_on='sh', right_on='sh_dc')

print(oc_station_df)
print(dc_station_df)

cols_to_use = dc_station_df.columns.difference(oc_station_df.columns)
merged_weather = pd.merge(oc_station_df, dc_station_df[cols_to_use], left_on='sh_oc', right_on='sh_dc', how='left')
print(merged_weather)
total_df = pd.concat([total_df, merged_weather])
total_df

      A          B sh  v1_oc  v2_oc sh_oc
0  Köln  Frankfurt  b     10      6     b
         A     B sh  v1_dc  v2_dc sh_dc
0  München  Köln  a      9      4     a
      A          B sh  v1_oc  v2_oc sh_oc sh_dc  v1_dc  v2_dc
0  Köln  Frankfurt  b     10      6     b   NaN    NaN    NaN


,A,B,sh,v1_oc,v2_oc,sh_oc,sh_dc,v1_dc,v2_dc
0,Köln,Frankfurt,b,10,6,b,NaN,NaN,NaN


In [100]:
total_df =  pd.DataFrame()
for st in stations_test:
    oc_station_df  = pd.merge(test[test['A'] == st], test_b.add_suffix('_oc'), how='left', 
                                left_on='sh', right_on='sh_oc')
    dc_station_df = pd.merge(test[test['B'] == st], test_b.add_suffix('_dc'), how='left', 
                                left_on='sh', right_on='sh_dc')

    cols_to_use = dc_station_df.columns.difference(oc_station_df.columns)
    merged_weather = pd.merge(oc_station_df, dc_station_df[cols_to_use], left_on='sh_oc', right_on='sh_dc', how='left')
    print(merged_weather)
    total_df = pd.concat([total_df, merged_weather])
total_df

         A     B sh  v1_oc  v2_oc sh_oc
0  München  Köln  a      9      4     a
           A        B sh  v1_dc  v2_dc sh_dc
0  Stuttgart  München  c      1     -1     c
         A     B sh  v1_oc  v2_oc sh_oc sh_dc  v1_dc  v2_dc
0  München  Köln  a      9      4     a   NaN    NaN    NaN
      A          B sh  v1_oc  v2_oc sh_oc
0  Köln  Frankfurt  b     10      6     b
         A     B sh  v1_dc  v2_dc sh_dc
0  München  Köln  a      9      4     a
      A          B sh  v1_oc  v2_oc sh_oc sh_dc  v1_dc  v2_dc
0  Köln  Frankfurt  b     10      6     b   NaN    NaN    NaN
           A        B sh  v1_oc  v2_oc sh_oc
0  Stuttgart  München  c      1     -1     c
           A          B sh  v1_dc  v2_dc sh_dc
0  Frankfurt  Stuttgart  d      2     -2     d
           A        B sh  v1_oc  v2_oc sh_oc sh_dc  v1_dc  v2_dc
0  Stuttgart  München  c      1     -1     c   NaN    NaN    NaN
           A          B sh  v1_oc  v2_oc sh_oc
0  Frankfurt  Stuttgart  d      2     -2     d
      A       

,A,B,sh,v1_oc,v2_oc,sh_oc,sh_dc,v1_dc,v2_dc
0,München,Köln,a,9,4,a,NaN,NaN,NaN
0,Köln,Frankfurt,b,10,6,b,NaN,NaN,NaN
0,Stuttgart,München,c,1,-1,c,NaN,NaN,NaN
0,Frankfurt,Stuttgart,d,2,-2,d,NaN,NaN,NaN


In [57]:
pd.merge(test[test['A'] == 'München'], test_b.add_suffix('_oc'), how='left', 
                             left_on='sh', right_on='sh_oc')

,A,B,sh,v1_oc,v2_oc,sh_oc


In [52]:
pd.merge(test[test['B'] == 3], test_b.add_suffix('_dc'), how='left', 
                             left_on='sh', right_on='sh_dc')

,A,B,sh,v1_dc,v2_dc,sh_dc
0,3,3,a,9,4,a
